In [1]:
import tensorflow as tf
import glob
import numpy as np

In [2]:
image_filenames = glob.glob('./dc/train/*.jpg')

In [3]:
lables = list(map(lambda x:x.split('\\')[1].split('.')[0],image_filenames))

In [4]:
train_lable = [[1,0] if x == 'cat' else [0,1] for x in lables]

In [5]:
image_que = tf.train.slice_input_producer([image_filenames,train_lable])

In [6]:
image_ = tf.read_file(image_que[0])
image = tf.image.decode_jpeg(image_, channels=3)

In [7]:
grey_image = tf.image.rgb_to_grayscale(image)
resize_image = tf.image.resize_images(grey_image,(200,200))
resize_image = tf.reshape(resize_image,[200,200,1])

In [8]:
new_img = tf.image.per_image_standardization(resize_image)

In [9]:
batch_size = 60
capacity = 10 + 2 * batch_size

In [10]:
image_batch,lable_batch = tf.train.batch([new_img,image_que[1]],batch_size=batch_size,capacity=capacity)

In [11]:
lable_batch.get_shape()

TensorShape([Dimension(60), Dimension(2)])

In [12]:
conv2d_1 = tf.contrib.layers.convolution2d(
    image_batch,
    num_outputs=32,
    weights_initializer=tf.truncated_normal_initializer(stddev=0.001),
    kernel_size = (5,5),
    activation_fn = tf.nn.relu,
    stride = (1,1),
    padding='SAME',
    trainable = True)
pool_1 = tf.nn.max_pool(conv2d_1,
                       ksize = [1,3,3,1],
                       strides = [1,2,2,1],
                       padding='SAME')

In [13]:
conv2d_2 = tf.contrib.layers.convolution2d(
    pool_1,
    num_outputs=32,
    weights_initializer=tf.truncated_normal_initializer(stddev=0.01),
    kernel_size = (5,5),
    activation_fn = tf.nn.relu,
    stride = (1,1),
    padding='SAME',
    trainable = True)
pool_2 = tf.nn.max_pool(conv2d_2,
                       ksize = [1,3,3,1],
                       strides = [1,2,2,1],
                       padding='SAME')

In [14]:
conv2d_3 = tf.contrib.layers.convolution2d(
    pool_2,
    num_outputs=64,
    weights_initializer=tf.truncated_normal_initializer(stddev=0.01),
    kernel_size = (5,5),
    activation_fn = tf.nn.relu,
    stride = (1,1),
    padding='SAME',
    trainable = True)
pool_3 = tf.nn.max_pool(conv2d_3,
                       ksize = [1,3,3,1],
                       strides = [1,2,2,1],
                       padding='SAME')

In [15]:
pool_3.get_shape()

TensorShape([Dimension(60), Dimension(25), Dimension(25), Dimension(64)])

In [16]:
pool3_flat = tf.reshape(pool_3, [-1, 25*25*64])
fc_1 = tf.contrib.layers.fully_connected(
                        pool3_flat,
                        1024,
                        weights_initializer=tf.truncated_normal_initializer(stddev=0.1),              
                        activation_fn = tf.nn.relu)

In [17]:
fc_2 = tf.contrib.layers.fully_connected(
                        fc_1,
                        192,
                        weights_initializer=tf.truncated_normal_initializer(stddev=0.1),              
                        activation_fn = tf.nn.relu)

In [18]:
#keep_prob = tf.placeholder("float")
#fc2_drop = tf.nn.dropout(fc_2, keep_prob)

In [19]:
out_w1 = tf.Variable(tf.truncated_normal([192,2]))
out_b1 = tf.Variable(tf.truncated_normal([2]))
comb_out = tf.matmul(fc_2,out_w1)+out_b1
pred = tf.sigmoid(comb_out)

In [20]:
pred.get_shape()

TensorShape([Dimension(60), Dimension(2)])

In [21]:
lable_batch.get_shape()

TensorShape([Dimension(60), Dimension(2)])

In [22]:
lable_batch = tf.cast(lable_batch,tf.float32)

In [23]:
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=lable_batch, logits=comb_out))

In [24]:
train_step = tf.train.AdamOptimizer(0.0001).minimize(loss)

In [25]:
predicted = tf.cast(pred>0.5,tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted,lable_batch),tf.float32))

In [26]:
saver = tf.train.Saver()
with tf.Session() as sess:
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    sess.run(tf.global_variables_initializer())
    for step in range(0,3000): 
        sess.run(train_step)
        if(step%100==0):
            res = sess.run([loss,accuracy])
            print(step,res)
            saver.save(sess,'./lesson30',global_step = step)
    coord.request_stop()
    coord.join(threads)

0 [0.77672142, 0.48333332]
100 [0.676386, 0.60833335]
200 [0.58545148, 0.64166665]
300 [0.69608992, 0.59166664]
400 [0.52750558, 0.74166667]
500 [0.49817747, 0.77499998]
600 [0.40623015, 0.80000001]
700 [0.4881697, 0.78333336]
800 [0.47797865, 0.80000001]
900 [0.30173245, 0.85833335]
1000 [0.3800492, 0.82499999]
1100 [0.44487202, 0.80000001]
1200 [0.32735005, 0.85833335]
1300 [0.33979523, 0.85833335]
1400 [0.19421081, 0.92500001]
1500 [0.22053032, 0.92500001]
1600 [0.27221364, 0.85833335]
1700 [0.089394137, 0.96666664]
1800 [0.11204717, 0.94999999]
1900 [0.13135758, 0.95833331]
2000 [0.14544156, 0.94166666]
2100 [0.01715141, 1.0]
INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.CancelledError'>, Enqueue operation was cancelled
	 [[Node: input_producer/input_producer/input_producer_EnqueueMany = QueueEnqueueManyV2[Tcomponents=[DT_INT32], timeout_ms=-1, _device="/job:localhost/replica:0/task:0/cpu:0"](input_producer/input_producer, input_prod

KeyboardInterrupt: 

In [ ]:
import os
ckpt = tf.train.get_checkpoint_state(os.path.dirname('__file__'))
saver = tf.train.Saver()
sess = tf.Session()
sess.run(tf.global_variables_initializer())
saver.restore(sess,ckpt.model_checkpoint_path)

INFO:tensorflow:Restoring parameters from lesson30-5900


In [ ]:
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(coord=coord,sess = sess)
for step in range(6000,7000): 
    sess.run(train_step,feed_dict={keep_prob: 0.5})
    if(step%100==0):
        res = sess.run([loss,accuracy],feed_dict={keep_prob: 1})
        print(step,res)
        saver.save(sess,'./lesson30',global_step = step)
coord.request_stop()
coord.join(threads)

6000 [0.32884693, 0.88999999]
6100 [0.40923646, 0.81999999]
6200 [0.34334183, 0.84500003]
6300 [0.32832718, 0.85000002]
6400 [0.29844311, 0.88]
6500 [0.44062489, 0.76999998]
